In [ ]:
!nvidia-smi

Sun Jul 26 19:55:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title Install Apex
# install apex to be able to use mix precision
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v -q --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [ ]:
!bash setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 7375 (delta 8), reused 0 (delta 0), pack-reused 7353
Receiving objects: 100% (7375/7375), 13.89 MiB | 28.79 MiB/s, done.
Resolving deltas: 100% (4979/4979), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Processing ./apex
Skipping wheel build for apex, due to binaries being disabled for it.
    Running setup.py install for apex ... done


In [ ]:
!pip install -U git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-agyu0u87
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-agyu0u87
     |████████████████████████████████| 3.0MB 3.4MB/s 
     |████████████████████████████████| 1.1MB 42.9MB/s 
     |████████████████████████████████| 890kB 41.7MB/s 
  Created wheel for transformers: filename=transformers-3.0.2-cp36-none-any.whl size=797698 sha256=4773209165b163982ab0c6c1d2d37af4b9e5f398bc61e05e41374983fd355d4a
  Stored in directory: /tmp/pip-ephem-wheel-cache-rrpqt4jo/wheels/33/eb/3b/4bf5dd835e865e472d4fc0754f35ac0edb08fe852e8f21655f
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=75162d72a170560bd64700d217fea4414d1d73926a317d1ecdf9a23bc80bc64e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built transformers sacremoses


## Dataset

In [ ]:
import logging
import os
import time
from dataclasses import dataclass, field
from enum import Enum
from typing import Dict, List, Optional, Union

import torch
from filelock import FileLock
from torch.utils.data.dataset import Dataset

from transformers.modeling_auto import MODEL_FOR_QUESTION_ANSWERING_MAPPING
from transformers.tokenization_utils import PreTrainedTokenizer
from transformers.data.processors.squad import SquadFeatures, SquadV1Processor, SquadV2Processor, squad_convert_examples_to_features
from transformers.data.data_collator import DataCollator


logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_FOR_QUESTION_ANSWERING_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)


@dataclass
class SquadDataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    model_type: str = field(
        default=None, metadata={"help": "Model type selected in the list: " + ", ".join(MODEL_TYPES)}
    )
    data_dir: str = field(
        default=None, metadata={"help": "The input data dir. Should contain the .json files for the SQuAD task."}
    )
    max_seq_length: int = field(
        default=128,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    doc_stride: int = field(
        default=128,
        metadata={"help": "When splitting up a long document into chunks, how much stride to take between chunks."},
    )
    max_query_length: int = field(
        default=64,
        metadata={
            "help": "The maximum number of tokens for the question. Questions longer than this will "
            "be truncated to this length."
        },
    )
    max_answer_length: int = field(
        default=30,
        metadata={
            "help": "The maximum length of an answer that can be generated. This is needed because the start "
            "and end predictions are not conditioned on one another."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    version_2_with_negative: bool = field(
        default=False, metadata={"help": "If true, the SQuAD examples contain some that do not have an answer."}
    )
    null_score_diff_threshold: float = field(
        default=0.0, metadata={"help": "If null_score - best_non_null is greater than the threshold predict null."}
    )
    n_best_size: int = field(
        default=20, metadata={"help": "If null_score - best_non_null is greater than the threshold predict null."}
    )
    lang_id: int = field(
        default=0,
        metadata={
            "help": "language id of input for language-specific xlm models (see tokenization_xlm.PRETRAINED_INIT_CONFIGURATION)"
        },
    )
    threads: int = field(default=1, metadata={"help": "multiple threads for converting example to features"})


class Split(Enum):
    train = "train"
    dev = "dev"


class SquadDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach
    soon.
    """

    args: SquadDataTrainingArguments
    features: List[SquadFeatures]
    mode: Split
    is_language_sensitive: bool

    def __init__(
        self,
        args: SquadDataTrainingArguments,
        tokenizer: PreTrainedTokenizer,
        limit_length: Optional[int] = None,
        mode: Union[str, Split] = Split.train,
        is_language_sensitive: Optional[bool] = False,
        cache_dir: Optional[str] = None,
    ):
        self.args = args
        self.is_language_sensitive = is_language_sensitive
        self.processor = SquadV2Processor() if args.version_2_with_negative else SquadV1Processor()
        if isinstance(mode, str):
            try:
                mode = Split[mode]
            except KeyError:
                raise KeyError("mode is not a valid split name")
        self.mode = mode
        # Load data features from cache or dataset file
        cached_features_file = os.path.join(
            cache_dir if cache_dir is not None else args.data_dir,
            "cached_{}_{}_{}".format(mode.value, tokenizer.__class__.__name__, str(args.max_seq_length),),
        )

        # Make sure only the first process in distributed training processes the dataset,
        # and the others will use the cache.
        lock_path = cached_features_file + ".lock"
        with FileLock(lock_path):
            if os.path.exists(cached_features_file) and not args.overwrite_cache:
                start = time.time()
                self.features = torch.load(cached_features_file)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )
            else:
                if mode == Split.dev:
                    examples = self.processor.get_dev_examples(args.data_dir)
                else:
                    examples = self.processor.get_train_examples(args.data_dir)

                self.features = squad_convert_examples_to_features(
                    examples=examples,
                    tokenizer=tokenizer,
                    max_seq_length=args.max_seq_length,
                    doc_stride=args.doc_stride,
                    max_query_length=args.max_query_length,
                    is_training=mode == Split.train,
                    threads=args.threads,
                )

                start = time.time()
                torch.save(self.features, cached_features_file)
                # ^ This seems to take a lot of time so I want to investigate why and how we can improve.
                logger.info(
                    "Saving features into cached file %s [took %.3f s]", cached_features_file, time.time() - start
                )

    def __len__(self):
        return len(self.features)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        # Convert to Tensors and build dataset
        feature = self.features[i]

        input_ids = torch.tensor(feature.input_ids, dtype=torch.long)
        attention_mask = torch.tensor(feature.attention_mask, dtype=torch.long)
        token_type_ids = torch.tensor(feature.token_type_ids, dtype=torch.long)
        cls_index = torch.tensor(feature.cls_index, dtype=torch.long)
        p_mask = torch.tensor(feature.p_mask, dtype=torch.float)
        is_impossible = torch.tensor(feature.is_impossible, dtype=torch.float)

        inputs = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
        }

        if self.args.model_type in ["xlm", "roberta", "distilbert", "camembert"]:
            del inputs["token_type_ids"]

        if self.args.model_type in ["xlnet", "xlm"]:
            inputs.update({"cls_index": cls_index, "p_mask": p_mask})
            if self.args.version_2_with_negative:
                inputs.update({"is_impossible": is_impossible})
            if self.is_language_sensitive:
                inputs.update({"langs": (torch.ones(input_ids.shape, dtype=torch.int64) * self.args.lang_id)})

        if self.mode == Split.train:
            start_positions = torch.tensor(feature.start_position, dtype=torch.long)
            end_positions = torch.tensor(feature.end_position, dtype=torch.long)
            inputs.update({"start_positions": start_positions, "end_positions": end_positions})

        return inputs


@dataclass
class DataCollatorForQuestionAnswering:
    """
    Data collator used for language modeling.
    - collates batches of tensors
    - preprocesses batches for question answering
    """

    def collate_batch(self, batch: List[Dict[str, torch.Tensor]]) -> Dict[str, torch.Tensor]:
        keys = batch[0].keys()
        inputs = {}

        for key in keys:
            inputs[key] = torch.stack([example[key] for example in batch])

        return inputs

## Write training script

In [ ]:
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, List, Optional

import numpy as np
import torch

from transformers import (
    ElectraConfig,
    ElectraForQuestionAnswering,
    ElectraTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)


logger = logging.getLogger(__name__)


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )


def main():
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    parser = HfArgumentParser((ModelArguments, SquadDataTrainingArguments, TrainingArguments))

    model_args, data_args, training_args = parser.parse_json_file(json_file='./args.json')

    if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
        )

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )
    logger.info("Training/evaluation parameters %s", training_args)

    # Prepare Question-Answering task
    # Load pretrained model and tokenizer
    #
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.

    config = ElectraConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
    tokenizer = ElectraTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
    model = ElectraForQuestionAnswering.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
    )

    # Get datasets
    is_language_sensitive = hasattr(model.config, "lang2id")
    train_dataset = (
        SquadDataset(
            data_args, tokenizer=tokenizer, is_language_sensitive=is_language_sensitive, cache_dir=model_args.cache_dir
        )
        if training_args.do_train
        else None
    )
    eval_dataset = (
        SquadDataset(
            data_args,
            tokenizer=tokenizer,
            mode="dev",
            is_language_sensitive=is_language_sensitive,
            cache_dir=model_args.cache_dir,
        )
        if training_args.do_eval
        else None
    )
    data_collator = DataCollatorForQuestionAnswering()

    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
    )

    # Training
    if training_args.do_train:
        trainer.train(
            model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
        )
        trainer.save_model()
        # For convenience, we also re-save the tokenizer to the same directory,
        # so that you can share your model easily on huggingface.co/models =)
        if trainer.is_world_master():
            tokenizer.save_pretrained(training_args.output_dir)


def _mp_fn(index):
    # For xla_spawn (TPUs)
    main()

## Train

In [ ]:
!mkdir squad_data
%cd squad_data
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
%cd ..

/content/squad_data
--2020-07-26 20:08:53--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.108.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30288272 (29M) [application/json]
Saving to: ‘train-v1.1.json’

train-v1.1.json     100%[===================>]  28.88M  47.0MB/s    in 0.6s    

2020-07-26 20:08:54 (47.0 MB/s) - ‘train-v1.1.json’ saved [30288272/30288272]

--2020-07-26 20:08:56--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.108.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4854279 (4.6M) [application/json]
Saving to: ‘dev-v1.1.json’



In [ ]:
import json

Let's write the arguments in a dict and store in a json file. The above code will load this file and parse the arguments.

In [ ]:
args_dict = {
  "model_name_or_path"          : "dbmdz/electra-base-turkish-cased-discriminator",
  "tokenizer_name"              : "dbmdz/electra-base-turkish-cased-discriminator",
  "model_type"                  : "electra",
  "data_dir"                    : "./squad_data",
  "max_seq_length"              : 512 ,
  "doc_stride"                  : 128,
  "fp16"                        : True,
  "max_grad_norm"               : 0.5,
  "per_device_train_batch_size" : 16,
  "per_device_eval_batch_size"  : 16,
  "gradient_accumulation_steps" : 8,
  "logging_steps"               : 50,
  "learning_rate"               : 3e-5,
  "num_train_epochs"            : 2,
  "overwrite_output_dir"        : True,
  "output_dir"                  : "./electra-base-discriminator-finetuned_squadv1_tr",
  "do_train"                    : True,
  "do_eval"                     : True
}

In [ ]:
with open('args.json', 'w') as f:
  json.dump(args_dict, f)

Start training!

In [ ]:
main()

07/26/2020 20:17:50 - INFO - transformers.training_args -   PyTorch: setting up devices
07/26/2020 20:17:50 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: True
07/26/2020 20:17:50 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='./electra-base-discriminator-finetuned_squadv1_tr', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, per_device_train_batch_size=16, per_device_eval_batch_size=16, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=8, learning_rate=3e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=0.5, num_train_epochs=2, max_steps=-1, warmup_steps=0, logging_dir='runs/Jul26_20-17-50_5ae585319611', logging_first_step=False, logging_steps=50, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=True, fp16_opt_level='O1', local_rank=-1, tpu_num_cores=None, tpu_m

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
07/26/2020 20:25:53 - INFO - transformers.trainer -   {'loss': 4.617739557027817, 'learning_rate': 2.1279069767441862e-05, 'epoch': 0.5797101449275363, 'step': 50}


07/26/2020 20:31:39 - INFO - transformers.trainer -   {'loss': 2.367682585120201, 'learning_rate': 1.2558139534883723e-05, 'epoch': 1.1623188405797102, 'step': 100}
07/26/2020 20:37:24 - INFO - transformers.trainer -   {'loss': 1.7186071726679801, 'learning_rate': 3.837209302325582e-06, 'epoch': 1.7420289855072464, 'step': 150}
07/26/2020 20:39:57 - INFO - transformers.trainer -   

Training completed. Do not forget to share your model on huggingface.co/models =)


07/26/2020 20:39:57 - INFO - transformers.trainer -   Saving model checkpoint to ./electra-base-discriminator-finetuned_squadv1_tr
07/26/2020 20:39:57 - INFO - transformers.configuration_utils -   Configuration saved in ./electra-base-discriminator-finetuned_squadv1_tr/config.json


07/26/2020 20:39:57 - INFO - transformers.modeling_utils -   Model weights saved in ./electra-base-discriminator-finetuned_squadv1_tr/pytorch_model.bin


## Eval

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/question-answering/run_squad.py

--2020-07-26 20:40:22--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/question-answering/run_squad.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34287 (33K) [text/plain]
Saving to: ‘run_squad.py’

run_squad.py        100%[===================>]  33.48K  --.-KB/s    in 0.01s   

2020-07-26 20:40:22 (2.80 MB/s) - ‘run_squad.py’ saved [34287/34287]



In [ ]:
!python run_squad.py \
--model_type electra \
--model_name_or_path electra-base-discriminator-finetuned_squadv1_tr \
--do_eval \
--data_dir ./squad_data \
--cache_dir ./squad_data \
--per_gpu_train_batch_size 16 \
--max_seq_length 512 \
--doc_stride 128 \
--output_dir ./squad_data \

2020-07-26 20:41:07.421800: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
07/26/2020 20:41:09 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
07/26/2020 20:41:09 - INFO - transformers.configuration_utils -   loading configuration file electra-base-discriminator-finetuned_squadv1_tr/config.json
07/26/2020 20:41:09 - INFO - transformers.configuration_utils -   Model config ElectraConfig {
  "_num_labels": 2,
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Load the fine-tuned model
tokenizer = AutoTokenizer.from_pretrained("./electra-base-discriminator-finetuned_squadv1_tr/")
model = AutoModelForQuestionAnswering.from_pretrained("./electra-base-discriminator-finetuned_squadv1_tr/")

07/26/2020 20:53:41 - INFO - transformers.configuration_utils -   loading configuration file ./electra-base-discriminator-finetuned_squadv1_tr/config.json
07/26/2020 20:53:41 - INFO - transformers.configuration_utils -   Model config ElectraConfig {
  "_num_labels": 2,
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "type_vocab_size": 2,
  "vocab_size": 32000
}

07/26/2020 20:53:41 - INFO - transformers.tokenization_utils_base -   Model name './electra-base-discriminator-finetu

## Model in action 🚀

In [ ]:

def ask_question(question,context):
  inputs = tokenizer.encode_plus(question, context, return_tensors="pt") 


  answer_start_scores, answer_end_scores = model(**inputs)
  answer_start = torch.argmax(answer_start_scores)  
  answer_end = torch.argmax(answer_end_scores) + 1  
  return tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

question = "Osmanlıların balkanlarda haçlılarla yaptığı ilk savaş hangisidir?"
context = """I.Murat 1326 yılında Bursada doğmuştur. I.Murat 1389 yılında Kosavada ölmüştür.
    Osmanlı İmparatorluğu'nun üçüncü padişahı olan I.Murat, 1359 ile 1389 yılları arasında beylik yapmıştır.
    I.Murat'ın babası Orhan Gazi, annesi Nilüfer Hatun'dur.
    I.Murat babası Orhan Gazi döneminde 95.000 kilometrekare olarak almış, devlet topraklarını I.Murat döneminde topraklarını yaklaşık 500.000 kilometrekareye genişlemiştir.
    1362 yılında Ankara Eratna beyliğinden yeniden alınmıştır.
    1362 yılında Sazlıdere savaşı ile gerçekleşmiştir. Sazlıdere savaşı sonucunda Edirne ve Filibe alınmıştır. Sazlıdere savaşı Osmanlı devleti ile Bizans ve Bulgar güçleri arasında gerçekleşmiştir.
    Hamitoğullarından Eğridir ve çevresi satın alındı.
    1364 yılında Sırpsındığı savaşı gerçekleşti. Balkan devletlerinden oluşan haçlı ordusunun başında Sırp Kralı I.Layoş vardı.
    Haçlı ordusu, Hacı İlbeyi tarafından yapılan ani bir baskın ile yok edilmiştir.
    Sırpsındığı savaşından sonra Edirne başkent yapılmıştır.
    Sırpsındığı savaşı Osmanlıların Balkanlarda haçlılarla yaptığı ilk savaştır.
    Bulgar krallığı Sırpsındığı savaşı sonrası Osmanlı Devletine bağlanmıştır.
    1371 yılında Çirmen savaşı gerçekleşti. Çirmen savaşı Evranos Bey ile Sırplar arasında olmuştur. Çirmen Savaşını Osmanlı Devleti kazanmıştır.
    Çirmen savaşı sonucunda Makedonya'nın bir kısmı alındı."""
ask_question(question,context)



'Sırpsındığı savaşı'

07/26/2020 20:43:09 - INFO - filelock -   Lock 139738508692504 acquired on /root/.cache/torch/transformers/de00d2114de85d1fbe7ae171c35b6b0118e2f7bfa9ecd55922d7bc6618439728.d465f032e0ca06e3c18d68488046720c2c4f07c049f9556caf0adf81cb7bdfb6.lock
07/26/2020 20:43:09 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/valhalla/electra-base-discriminator-finetuned_squadv1/config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp6886k0t0


07/26/2020 20:43:09 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/valhalla/electra-base-discriminator-finetuned_squadv1/config.json in cache at /root/.cache/torch/transformers/de00d2114de85d1fbe7ae171c35b6b0118e2f7bfa9ecd55922d7bc6618439728.d465f032e0ca06e3c18d68488046720c2c4f07c049f9556caf0adf81cb7bdfb6
07/26/2020 20:43:09 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/de00d2114de85d1fbe7ae171c35b6b0118e2f7bfa9ecd55922d7bc6618439728.d465f032e0ca06e3c18d68488046720c2c4f07c049f9556caf0adf81cb7bdfb6
07/26/2020 20:43:09 - INFO - filelock -   Lock 139738508692504 released on /root/.cache/torch/transformers/de00d2114de85d1fbe7ae171c35b6b0118e2f7bfa9ecd55922d7bc6618439728.d465f032e0ca06e3c18d68488046720c2c4f07c049f9556caf0adf81cb7bdfb6.lock
07/26/2020 20:43:09 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/valhalla/e

07/26/2020 20:43:10 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/valhalla/electra-base-discriminator-finetuned_squadv1/vocab.txt in cache at /root/.cache/torch/transformers/6dff6cfcf5d5158789294447f218eaceeb3400e34d5da8056818a03aaa63ed26.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
07/26/2020 20:43:10 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/6dff6cfcf5d5158789294447f218eaceeb3400e34d5da8056818a03aaa63ed26.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
07/26/2020 20:43:10 - INFO - filelock -   Lock 139738508692504 released on /root/.cache/torch/transformers/6dff6cfcf5d5158789294447f218eaceeb3400e34d5da8056818a03aaa63ed26.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


07/26/2020 20:43:10 - INFO - filelock -   Lock 139739108966696 acquired on /root/.cache/torch/transformers/94e96e4fc3b8d300704e845d5ebf2c9aa77027b3cc889a912b8bec3ecba5285d.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4.lock
07/26/2020 20:43:10 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/valhalla/electra-base-discriminator-finetuned_squadv1/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp4fnc1izx


07/26/2020 20:43:10 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/valhalla/electra-base-discriminator-finetuned_squadv1/special_tokens_map.json in cache at /root/.cache/torch/transformers/94e96e4fc3b8d300704e845d5ebf2c9aa77027b3cc889a912b8bec3ecba5285d.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4
07/26/2020 20:43:10 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/94e96e4fc3b8d300704e845d5ebf2c9aa77027b3cc889a912b8bec3ecba5285d.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4
07/26/2020 20:43:10 - INFO - filelock -   Lock 139739108966696 released on /root/.cache/torch/transformers/94e96e4fc3b8d300704e845d5ebf2c9aa77027b3cc889a912b8bec3ecba5285d.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4.lock
07/26/2020 20:43:10 - INFO - filelock -   Lock 139738508692504 acquired on /root/.cache/torch/transformers/c9d8dd05ed2da6fe24bfd86a078c0e3d320e

07/26/2020 20:43:11 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/valhalla/electra-base-discriminator-finetuned_squadv1/tokenizer_config.json in cache at /root/.cache/torch/transformers/c9d8dd05ed2da6fe24bfd86a078c0e3d320e36d4f5cd912a84c89727979a96fd.11f57497ee659e26f830788489816dbcb678d91ae48c06c50c9dc0e4438ec05b
07/26/2020 20:43:11 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/c9d8dd05ed2da6fe24bfd86a078c0e3d320e36d4f5cd912a84c89727979a96fd.11f57497ee659e26f830788489816dbcb678d91ae48c06c50c9dc0e4438ec05b
07/26/2020 20:43:11 - INFO - filelock -   Lock 139738508692504 released on /root/.cache/torch/transformers/c9d8dd05ed2da6fe24bfd86a078c0e3d320e36d4f5cd912a84c89727979a96fd.11f57497ee659e26f830788489816dbcb678d91ae48c06c50c9dc0e4438ec05b.lock
07/26/2020 20:43:11 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/valhalla/e

07/26/2020 20:43:11 - INFO - transformers.modelcard -   Model card: {
  "caveats_and_recommendations": {},
  "ethical_considerations": {},
  "evaluation_data": {},
  "factors": {},
  "intended_use": {},
  "metrics": {},
  "model_details": {},
  "quantitative_analyses": {},
  "training_data": {}
}

07/26/2020 20:43:11 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/valhalla/electra-base-discriminator-finetuned_squadv1/config.json from cache at /root/.cache/torch/transformers/de00d2114de85d1fbe7ae171c35b6b0118e2f7bfa9ecd55922d7bc6618439728.d465f032e0ca06e3c18d68488046720c2c4f07c049f9556caf0adf81cb7bdfb6
07/26/2020 20:43:11 - INFO - transformers.configuration_utils -   Model config ElectraConfig {
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_rang

07/26/2020 20:43:17 - INFO - transformers.file_utils -   storing https://cdn.huggingface.co/valhalla/electra-base-discriminator-finetuned_squadv1/pytorch_model.bin in cache at /root/.cache/torch/transformers/5fb628232118635dacd5ef6daa80b4fac5976ed02a8e05e650a88c32d482d889.4d5270b238ee40016eeb0c0975da2e2da6b229906da3b937b0a82561fc0210da
07/26/2020 20:43:17 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/5fb628232118635dacd5ef6daa80b4fac5976ed02a8e05e650a88c32d482d889.4d5270b238ee40016eeb0c0975da2e2da6b229906da3b937b0a82561fc0210da
07/26/2020 20:43:17 - INFO - filelock -   Lock 139738594959432 released on /root/.cache/torch/transformers/5fb628232118635dacd5ef6daa80b4fac5976ed02a8e05e650a88c32d482d889.4d5270b238ee40016eeb0c0975da2e2da6b229906da3b937b0a82561fc0210da.lock
07/26/2020 20:43:17 - INFO - transformers.modeling_utils -   loading weights file https://cdn.huggingface.co/valhalla/electra-base-discriminator-finetuned_squadv1/pytorch_mod

07/26/2020 20:43:20 - INFO - transformers.modeling_utils -   All model checkpoint weights were used when initializing ElectraForQuestionAnswering.

07/26/2020 20:43:20 - WARNING - transformers.modeling_utils -   Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at valhalla/electra-base-discriminator-finetuned_squadv1 and are newly initialized: ['electra.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
